In [1]:
import numpy as np
import pandas as pd

In [2]:
data1=pd.read_csv("D:\\Final Year Project\\FML\\2nd Model\\preprocessed_XIIoT_ID.csv")

In [3]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 596022 entries, 0 to 596021
Data columns (total 79 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   Timestamp                         596022 non-null  float64
 1   Scr_port                          596022 non-null  int64  
 2   Des_port                          596022 non-null  int64  
 3   Duration                          596022 non-null  float64
 4   Scr_bytes                         596022 non-null  int64  
 5   Des_bytes                         596022 non-null  int64  
 6   Conn_state                        596022 non-null  int64  
 7   missed_bytes                      596022 non-null  int64  
 8   is_syn_only                       596022 non-null  bool   
 9   Is_SYN_ACK                        596022 non-null  bool   
 10  is_pure_ack                       596022 non-null  bool   
 11  is_with_payload                   596022 non-null  b

In [28]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
from skfeature.function.information_theoretical_based import MRMR
from sklearn.preprocessing import LabelEncoder

Label Encoding

In [29]:
column_to_encode = 'Protocol'

# Create a LabelEncoder instance
label_encoder = LabelEncoder()

# Label encode the 'protocol' column
data1[column_to_encode + '_encoded'] = label_encoder.fit_transform(data1[column_to_encode])

# Display the result
print(data1.head())

         Date   Timestamp         Scr_IP Scr_port        Des_IP Des_port  \
0   9/01/2020  1578540956  192.168.2.199    49278  192.168.2.10       80   
1  13/01/2020  1578871873       10.0.1.5    39769  131.236.3.92       53   
2   9/01/2020  1578522486    172.24.1.80    59050    172.24.1.1       53   
3  27/02/2020  1582757640  192.168.2.196    37966  192.168.2.10     1880   
4  16/12/2019  1576452612    172.24.1.80    38233    172.24.1.1       53   

  Protocol    Service  Duration Scr_bytes Des_bytes  Conn_state missed_bytes  \
0      tcp       http   0.67369     13437     34924           1            0   
1      udp        dns  0.000083        78         0           1            0   
2      udp        dns  0.000132        38        38           1            0   
3      tcp  websocket  9.378481      1121       484           1            0   
4      udp        dns  0.000074         -         -           1            0   

   is_syn_only  Is_SYN_ACK  is_pure_ack  is_with_payload  FIN 

One-Hot Encoding

In [30]:
column_to_encode = 'Service'

# Perform one-hot encoding directly in the same DataFrame
data1 = pd.get_dummies(data1, columns=[column_to_encode], prefix=[column_to_encode])

# Convert True/False to 1/0 in the one-hot encoded columns
data1[data1.filter(like=column_to_encode).columns] = data1[data1.filter(like=column_to_encode).columns].astype(int)

# Display the updated DataFrame
print(data1.head())

         Date   Timestamp         Scr_IP Scr_port        Des_IP Des_port  \
0   9/01/2020  1578540956  192.168.2.199    49278  192.168.2.10       80   
1  13/01/2020  1578871873       10.0.1.5    39769  131.236.3.92       53   
2   9/01/2020  1578522486    172.24.1.80    59050    172.24.1.1       53   
3  27/02/2020  1582757640  192.168.2.196    37966  192.168.2.10     1880   
4  16/12/2019  1576452612    172.24.1.80    38233    172.24.1.1       53   

  Protocol  Duration Scr_bytes Des_bytes  Conn_state missed_bytes  \
0      tcp   0.67369     13437     34924           1            0   
1      udp  0.000083        78         0           1            0   
2      udp  0.000132        38        38           1            0   
3      tcp  9.378481      1121       484           1            0   
4      udp  0.000074         -         -           1            0   

   is_syn_only  Is_SYN_ACK  is_pure_ack  is_with_payload  FIN or RST  \
0         True        True         True             True

In [34]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 820834 entries, 0 to 820833
Data columns (total 85 columns):
 #   Column                            Non-Null Count   Dtype 
---  ------                            --------------   ----- 
 0   Date                              820503 non-null  object
 1   Timestamp                         820537 non-null  object
 2   Scr_IP                            820834 non-null  object
 3   Scr_port                          820834 non-null  object
 4   Des_IP                            820834 non-null  object
 5   Des_port                          820834 non-null  object
 6   Protocol                          820834 non-null  object
 7   Duration                          820834 non-null  object
 8   Scr_bytes                         820834 non-null  object
 9   Des_bytes                         820834 non-null  object
 10  Conn_state                        820834 non-null  int64 
 11  missed_bytes                      820834 non-null  object
 12  is

Converting column Anomaly alert to int

In [33]:
# Replace 'TRUE' with 1, 'FALSE' with 0, and other non-numeric values with NaN
data1['anomaly_alert'] = pd.to_numeric(data1['anomaly_alert'].replace({'TRUE': 1, 'FALSE': 0, '-': np.nan}), errors='coerce')

# Convert the column to integers
data1['anomaly_alert'] = data1['anomaly_alert'].astype(float).astype('Int64')

Converting Target label class3 to numeric

In [35]:
# Replace 'TRUE' with 1, 'FALSE' with 0, and other non-numeric values with NaN
data1['class3'] = pd.to_numeric(data1['class3'].replace({'Attack': 1, 'Normal': 0, '-': np.nan}), errors='coerce')

# Convert the column to integers
data1['class3'] = data1['class3'].astype(float).astype('Int64')

C:\Users\Varshini S\AppData\Local\Temp\ipykernel_24264\1149340654.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data1['class3'] = pd.to_numeric(data1['class3'].replace({'Attack': 1, 'Normal': 0, '-': np.nan}), errors='coerce')


Drop unwanted columns and store in data2

In [36]:
data_to_remove = data1[['Date', 'Scr_IP', 'Des_IP','Protocol','class1','class2']]  # Replace with actual column names

# Create a new DataFrame without the specified columns
data2 = data1.drop(columns=['Date', 'Scr_IP', 'Des_IP','Protocol','class1','class2'])  # Replace with actual column names

# Display the original DataFrame and the new DataFrame

In [37]:
data2

,Timestamp,Scr_port,Des_port,Duration,Scr_bytes,Des_bytes,Conn_state,missed_bytes,is_syn_only,Is_SYN_ACK,is_pure_ack,is_with_payload,FIN or RST,Bad_checksum,is_SYN_with_RST,Scr_pkts,Scr_ip_bytes,Des_pkts,Des_ip_bytes,anomaly_alert,total_bytes,total_packet,paket_rate,byte_rate,Scr_packts_ratio,Des_pkts_ratio,Scr_bytes_ratio,Des_bytes_ratio,Avg_user_time,Std_user_time,Avg_nice_time,Std_nice_time,Avg_system_time,Std_system_time,Avg_iowait_time,Std_iowait_time,Avg_ideal_time,Std_ideal_time,Avg_tps,Std_tps,Avg_rtps,Std_rtps,Avg_wtps,Std_wtps,Avg_ldavg_1,Std_ldavg_1,Avg_kbmemused,Std_kbmemused,Avg_num_Proc/s,Std_num_proc/s,Avg_num_cswch/s,std_num_cswch/s,OSSEC_alert,OSSEC_alert_level,Login_attempt,Succesful_login,File_activity,Process_activity,read_write_physical.process,is_privileged,class3,Protocol_encoded,Service_coap,Service_dhcp,Service_dns,Service_echo,Service_http,Service_https,Service_imap,Service_modbus,Service_mqtt,Service_mysql,Service_netbios-ns,Service_other,Service_private,Service_simple_service_discovery,Service_smtp,Service_ssh,Service_websocket
0,1578540956,49278,80,0.67369,13437,34924,1,0,True,True,True,True,True,False,False,105,18905,105,40392,1,107658,210,311.7160712,159803.4704,0.5,0.5,0.300414275,0.699585725,9.207,5.55584206,10.994,1.356305275,4.864,1.873004004,0.311,0.224653066,74.624,8.245611196,12.297,10.38585004,8,10.50714043,4.297,2.723578712,2.146,0.102781321,915852.8,2507.97563,5.1,3.238826948,2806.2,158.7493622,1,5,0,0,0,0,0,0,1,2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,1578871873,39769,53,0.000083,78,0,1,0,False,False,False,True,False,False,False,2,134,0,0,0,212,2,24096.38554,2554216.867,1,0,1,0,12.326,6.674247823,2.895,1.604302029,3.778,2.099741889,2.741,5.327471164,78.263,8.869399134,6.6,5.713142743,0,0,6.6,5.713142743,0.612,0.037629775,910498.4,4044.112046,1,0.894427191,1561.6,247.0802299,0,0,0,0,0,0,0,0,0,3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1578522486,59050,53,0.000132,38,38,1,0,False,False,False,True,False,False,False,1,66,1,66,0,208,2,15151.51515,1575757.576,0.5,0.5,0.5,0.5,6.931,6.416007248,0.706,0.408905857,1.693,0.771635277,2.423,3.829809525,88.245,7.112108337,37.4,40.19004852,30.1,39.79811553,7.3,3.1,0.55,0.02,921020.4,2139.652645,1,0,1603.3,294.1390997,0,0,0,0,0,0,0,0,0,3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1582757640,37966,1880,9.378481,1121,484,1,0,True,True,True,True,True,False,False,8,1545,6,804,0,3954,14,1.492779055,421.6034558,0.571428571,0.428571429,0.67425392,0.32574608,10.244,7.932040343,1.965,1.514168088,3.941,1.709055002,0.509,0.425522032,83.339,10.20608686,29,26.57442379,23.3,26.12680616,5.7,6.229767251,0.708,0.026,915874.8,977.8851466,7.2,5.325410782,1685.3,467.8737116,0,0,1,1,1,1,1,1,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,1576452612,38233,53,0.000074,-,-,1,0,False,False,False,True,False,False,False,-,-,-,-,0,-,-,-,-,-,-,-,-,5.181,5.980458929,2.365,1.284462923,2.415,1.401151312,1.803,4.478256469,88.239,8.090017862,9.8,1.326649916,0.1,0.3,9.7,1.268857754,0.536,0.030066593,918213.2,2342.93836,0.6,0.489897949,1456.5,346.8847791,0,0,0,0,0,0,0,0,0,3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
820829,1578374008.0,50886,22,-,-,-,0,0,False,False,False,False,False,False,False,1,60,0,0,1,-,1,-,-,1,0,-,-,0.957,0.634618783,1.515,0.814103802,1.563,0.838022076,1.506,3.449217882,94.463,3.776329038,18,9.654014709,6.3,7.849203781,11.7,3.874274126,0.616,0.083809307,890402,270.7338176,0.9,0.538516481,1315.7,461.6054701,0,0,0,0,0,0,0,0,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
820830,1579653575.0,45913,1880,63.290477,0,0,1,0,True,True,True,False,True,False,False,4,216,3,164,0,380,7,0.110601157,6.004062823,0.571428571,0.428571429,0.568421053,0.431578947,10.343,11.60153787,2.011,1.848531579,2.823,1.78

Handling Missing Values

In [38]:
data2.replace('-', np.nan, inplace=True)

In [41]:
len(data2)

820834

In [40]:
data3 = data2.dropna().copy()

In [42]:
len(data3)

596543

In [43]:
pd.set_option('display.max_columns', None)  # Show all columns
print(data3.head())

    Timestamp Scr_port Des_port  Duration Scr_bytes Des_bytes  Conn_state  \
0  1578540956    49278       80   0.67369     13437     34924           1   
1  1578871873    39769       53  0.000083        78         0           1   
2  1578522486    59050       53  0.000132        38        38           1   
3  1582757640    37966     1880  9.378481      1121       484           1   
5  1580452611    50502       80  5.153928       460       671           1   

  missed_bytes  is_syn_only  Is_SYN_ACK  is_pure_ack  is_with_payload  \
0            0         True        True         True             True   
1            0        False       False        False             True   
2            0        False       False        False             True   
3            0         True        True         True             True   
5            0         True        True         True             True   

   FIN or RST  Bad_checksum  is_SYN_with_RST Scr_pkts Scr_ip_bytes Des_pkts  \
0        True      

Drop Infinite values

In [51]:
non_finite_values = ~pd.to_numeric(data3['Avg_user_time'], errors='coerce').notna()
print(data3[non_finite_values])

Empty DataFrame
Columns: [Timestamp, Scr_port, Des_port, Duration, Scr_bytes, Des_bytes, Conn_state, missed_bytes, is_syn_only, Is_SYN_ACK, is_pure_ack, is_with_payload, FIN or RST, Bad_checksum, is_SYN_with_RST, Scr_pkts, Scr_ip_bytes, Des_pkts, Des_ip_bytes, anomaly_alert, total_bytes, total_packet, paket_rate, byte_rate, Scr_packts_ratio, Des_pkts_ratio, Scr_bytes_ratio, Des_bytes_ratio, Avg_user_time, Std_user_time, Avg_nice_time, Std_nice_time, Avg_system_time, Std_system_time, Avg_iowait_time, Std_iowait_time, Avg_ideal_time, Std_ideal_time, Avg_tps, Std_tps, Avg_rtps, Std_rtps, Avg_wtps, Std_wtps, Avg_ldavg_1, Std_ldavg_1, Avg_kbmemused, Std_kbmemused, Avg_num_Proc/s, Std_num_proc/s, Avg_num_cswch/s, std_num_cswch/s, OSSEC_alert, OSSEC_alert_level, Login_attempt, Succesful_login, File_activity, Process_activity, read_write_physical.process, is_privileged, class3, Protocol_encoded, Service_coap, Service_dhcp, Service_dns, Service_echo, Service_http, Service_https, Service_imap, S

In [50]:
column_name = 'Avg_user_time'

# Identify rows with non-finite values
non_finite_values = ~pd.to_numeric(data3[column_name], errors='coerce').notna()

# Drop rows with non-finite values from the same DataFrame
data3.drop(data3[non_finite_values].index, inplace=True)


Convert Object to Integer

In [ ]:
data3['Std_kbmemused'] = pd.to_numeric(data3['Std_kbmemused'], errors='coerce').astype(int)

In [47]:
# Assuming 'data3_cleaned' is your DataFrame
columns_to_convert = ['Timestamp', 'Scr_port', 'Des_port']  # Replace with your column names

# Convert selected columns to numeric
data3[columns_to_convert] = data3[columns_to_convert].apply(pd.to_numeric, errors='coerce')




In [52]:
columns_to_convert = ['Duration', 'Scr_bytes', 'Des_bytes','missed_bytes','Scr_pkts','Scr_ip_bytes','Des_pkts','Des_ip_bytes','total_bytes','total_packet','paket_rate','byte_rate','Scr_packts_ratio','Des_pkts_ratio','Scr_bytes_ratio','Des_bytes_ratio','Avg_user_time','Std_user_time','Avg_nice_time','Std_nice_time']  # Replace with your column names

# Convert selected columns to numeric
data3[columns_to_convert] = data3[columns_to_convert].apply(pd.to_numeric, errors='coerce')

In [54]:
columns_to_convert = ['Avg_system_time', 'Std_system_time', 'Avg_iowait_time','Std_iowait_time','Avg_ideal_time','Std_ideal_time','Avg_tps','Std_tps','Avg_rtps','Std_rtps','Avg_wtps','Std_wtps','Avg_ldavg_1','Std_ldavg_1','Avg_kbmemused','Std_kbmemused','Avg_num_Proc/s','Std_num_proc/s','Avg_num_cswch/s','std_num_cswch/s']  # Replace with your column names

# Convert selected columns to numeric
data3[columns_to_convert] = data3[columns_to_convert].apply(pd.to_numeric, errors='coerce')

In [55]:
data3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 596022 entries, 0 to 820833
Data columns (total 79 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   Timestamp                         596022 non-null  float64
 1   Scr_port                          596022 non-null  int64  
 2   Des_port                          596022 non-null  int64  
 3   Duration                          596022 non-null  float64
 4   Scr_bytes                         596022 non-null  int64  
 5   Des_bytes                         596022 non-null  int64  
 6   Conn_state                        596022 non-null  int64  
 7   missed_bytes                      596022 non-null  int64  
 8   is_syn_only                       596022 non-null  bool   
 9   Is_SYN_ACK                        596022 non-null  bool   
 10  is_pure_ack                       596022 non-null  bool   
 11  is_with_payload                   596022 non-null  bool  

In [56]:
data3

,Timestamp,Scr_port,Des_port,Duration,Scr_bytes,Des_bytes,Conn_state,missed_bytes,is_syn_only,Is_SYN_ACK,is_pure_ack,is_with_payload,FIN or RST,Bad_checksum,is_SYN_with_RST,Scr_pkts,Scr_ip_bytes,Des_pkts,Des_ip_bytes,anomaly_alert,total_bytes,total_packet,paket_rate,byte_rate,Scr_packts_ratio,Des_pkts_ratio,Scr_bytes_ratio,Des_bytes_ratio,Avg_user_time,Std_user_time,Avg_nice_time,Std_nice_time,Avg_system_time,Std_system_time,Avg_iowait_time,Std_iowait_time,Avg_ideal_time,Std_ideal_time,Avg_tps,Std_tps,Avg_rtps,Std_rtps,Avg_wtps,Std_wtps,Avg_ldavg_1,Std_ldavg_1,Avg_kbmemused,Std_kbmemused,Avg_num_Proc/s,Std_num_proc/s,Avg_num_cswch/s,std_num_cswch/s,OSSEC_alert,OSSEC_alert_level,Login_attempt,Succesful_login,File_activity,Process_activity,read_write_physical.process,is_privileged,class3,Protocol_encoded,Service_coap,Service_dhcp,Service_dns,Service_echo,Service_http,Service_https,Service_imap,Service_modbus,Service_mqtt,Service_mysql,Service_netbios-ns,Service_other,Service_private,Service_simple_service_discovery,Service_smtp,Service_ssh,Service_websocket
0,1.578541e+09,49278,80,0.673690,13437,34924,1,0,True,True,True,True,True,False,False,105,18905,105,40392,1,107658,210,311.716071,1.598035e+05,0.500000,0.500000,0.300414,0.699586,9.207,5.555842,10.994,1.356305,4.864,1.873004,0.311,0.224653,74.624,8.245611,12.297,10.385850,8.000,10.507140,4.297,2.723579,2.146,0.102781,915852.8,2507.975630,5.100,3.238827,2806.200,158.749362,1,5,0,0,0,0,0,0,1,2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,1.578872e+09,39769,53,0.000083,78,0,1,0,False,False,False,True,False,False,False,2,134,0,0,0,212,2,24096.385540,2.554217e+06,1.000000,0.000000,1.000000,0.000000,12.326,6.674248,2.895,1.604302,3.778,2.099742,2.741,5.327471,78.263,8.869399,6.600,5.713143,0.000,0.000000,6.600,5.713143,0.612,0.037630,910498.4,4044.112046,1.000,0.894427,1561.600,247.080230,0,0,0,0,0,0,0,0,0,3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.578522e+09,59050,53,0.000132,38,38,1,0,False,False,False,True,False,False,False,1,66,1,66,0,208,2,15151.515150,1.575758e+06,0.500000,0.500000,0.500000,0.500000,6.931,6.416007,0.706,0.408906,1.693,0.771635,2.423,3.829810,88.245,7.112108,37.400,40.190049,30.100,39.798116,7.300,3.100000,0.550,0.020000,921020.4,2139.652645,1.000,0.000000,1603.300,294.139100,0,0,0,0,0,0,0,0,0,3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1.582758e+09,37966,1880,9.378481,1121,484,1,0,True,True,True,True,True,False,False,8,1545,6,804,0,3954,14,1.492779,4.216035e+02,0.571429,0.428571,0.674254,0.325746,10.244,7.932040,1.965,1.514168,3.941,1.709055,0.509,0.425522,83.339,10.206087,29.000,26.574424,23.300,26.126806,5.700,6.229767,0.708,0.026000,915874.8,977.885147,7.200,5.325411,1685.300,467.873712,0,0,1,1,1,1,1,1,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
5,1.580453e+09,50502,80,5.153928,460,671,1,0,True,True,True,True,True,False,False,12,1560,10,1878,0,4569,22,4.268589,8.865083e+02,0.545455,0.454545,0.442110,0.557890,10.579,6.866914,3.273,2.366141,2.638,1.040921,0.279,0.210829,83.233,9.579212,19.300,18.423083,8.800,10.205881,10.500,13.529597,0.524,0.028000,910370.8,4006.881700,0.900,0.538516,1366.900,304.062642,0,0,0,0,0,0,1,0,0,2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
820827,1.579041e+09,51292,80,0.205832,127,310,1,0,True,True,True,True,True,False,False,6,447,5,578,0,1462,11,53.441642,7.102880e+03,0.545455,0.454545,0.392613,0.607387,9.732,4.592635,8.242,2.850375,6.224,1.459577,0.307,0.358303,75.494,7.757618,25.827,26.899018,17.424,18.935733,8.403,13.268159,1.647,0.050210,918959.6,1914.446510,0.900,0.538516,2676.400,483.980000,1,5,0,0,0,0,0,0,1,2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
820830,1.579654e+09,45913,1880,63.290477,0,0,1,0,True,True,True,False,True,False,False,4,216,3,164,0,

Seperating Target Variable

In [6]:
data1=data1.dropna()

In [7]:
X=data1.drop(columns=['class3'])

In [8]:
y=data1.iloc[:,60]

In [9]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 596018 entries, 0 to 596021
Data columns (total 78 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   Timestamp                         596018 non-null  float64
 1   Scr_port                          596018 non-null  int64  
 2   Des_port                          596018 non-null  int64  
 3   Duration                          596018 non-null  float64
 4   Scr_bytes                         596018 non-null  int64  
 5   Des_bytes                         596018 non-null  int64  
 6   Conn_state                        596018 non-null  int64  
 7   missed_bytes                      596018 non-null  int64  
 8   is_syn_only                       596018 non-null  bool   
 9   Is_SYN_ACK                        596018 non-null  bool   
 10  is_pure_ack                       596018 non-null  bool   
 11  is_with_payload                   596018 non-null  b

In [10]:
y.info()

<class 'pandas.core.series.Series'>
Int64Index: 596018 entries, 0 to 596021
Series name: class3
Non-Null Count   Dtype
--------------   -----
596018 non-null  int64
dtypes: int64(1)
memory usage: 9.1 MB


In [64]:
# Assuming 'preprocessed_data' is your preprocessed DataFrame
data3.to_csv('preprocessed_XIIoT_ID.csv', index=False)


FEATURE SELECTION WITH RFE

In [72]:
# Initialize the Random Forest classifier
rf_classifier = RandomForestClassifier()

# Perform Recursive Feature Elimination (RFE)
rfe = RFE(rf_classifier)  # 'k' is the number of features you want to select
rfe.fit(X, y)

# Get the selected features using RFE
selected_features_rfe=X.iloc[:,rfe.support_]

selected_features_rfe.info()

<class 'pandas.core.frame.DataFrame'>
Index: 596018 entries, 0 to 820833
Data columns (total 39 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Timestamp                    596018 non-null  float64
 1   Scr_port                     596018 non-null  int64  
 2   Des_port                     596018 non-null  int64  
 3   Duration                     596018 non-null  float64
 4   Scr_bytes                    596018 non-null  int64  
 5   Des_bytes                    596018 non-null  int64  
 6   Conn_state                   596018 non-null  int64  
 7   is_syn_only                  596018 non-null  bool   
 8   is_with_payload              596018 non-null  bool   
 9   FIN or RST                   596018 non-null  bool   
 10  Scr_pkts                     596018 non-null  int64  
 11  Scr_ip_bytes                 596018 non-null  int64  
 12  Des_pkts                     596018 non-null  int64  
 13  Des_

In [11]:
array=np.array(["Timestamp","Scr_port","Des_port","Duration","Scr_bytes","Des_bytes","Conn_state","is_syn_only","is_with_payload","FIN or RST","Scr_pkts","Scr_ip_bytes","Des_pkts","Des_ip_bytes","anomaly_alert","total_bytes","total_packet","paket_rate","byte_rate","Scr_packts_ratio","Des_pkts_ratio","Scr_bytes_ratio","Des_bytes_ratio","Avg_user_time","Avg_nice_time","Avg_system_time","Avg_ideal_time","Avg_num_cswch/s","std_num_cswch/s","OSSEC_alert_level","Login_attempt","Process_activity","read_write_physical.process","Protocol_encoded","Service_coap","Service_dns","Service_modbus","Service_mqtt","Service_other"])

In [12]:
selected_features_rfe=X.loc[:,array]

In [13]:
selected_features_rfe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 596018 entries, 0 to 596021
Data columns (total 39 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Timestamp                    596018 non-null  float64
 1   Scr_port                     596018 non-null  int64  
 2   Des_port                     596018 non-null  int64  
 3   Duration                     596018 non-null  float64
 4   Scr_bytes                    596018 non-null  int64  
 5   Des_bytes                    596018 non-null  int64  
 6   Conn_state                   596018 non-null  int64  
 7   is_syn_only                  596018 non-null  bool   
 8   is_with_payload              596018 non-null  bool   
 9   FIN or RST                   596018 non-null  bool   
 10  Scr_pkts                     596018 non-null  int64  
 11  Scr_ip_bytes                 596018 non-null  int64  
 12  Des_pkts                     596018 non-null  int64  
 13 

FEATURE SELECTION WITH MRMR

In [17]:
from skfeature.function.information_theoretical_based import MRMR
selected_features_mrmr = MRMR.mrmr(X.values, y.values)

In [18]:
selected_features_mrmr[0:39]

array([54, 64, 63,  7, 56, 46, 59, 26, 36, 23, 76, 11, 58, 67, 37, 70, 74,
       69, 65, 57,  6, 43, 60, 28, 30,  1, 22,  8,  2,  5, 24, 20, 77, 51,
       14, 42, 33, 55, 40])

In [15]:
mrmr_data=X.iloc[:,[54, 64, 63,  7, 56, 46, 59, 26, 36, 23, 76, 11, 58, 67, 37, 70, 74,
       69, 65, 57,  6, 43, 60, 28, 30,  1, 22,  8,  2,  5, 24, 20, 77, 51,
       14, 42, 33, 55, 40]]

In [16]:
mrmr_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 596018 entries, 0 to 596021
Data columns (total 39 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   Login_attempt                     596018 non-null  int64  
 1   Service_echo                      596018 non-null  int64  
 2   Service_dns                       596018 non-null  int64  
 3   missed_bytes                      596018 non-null  int64  
 4   File_activity                     596018 non-null  int64  
 5   Avg_kbmemused                     596018 non-null  float64
 6   is_privileged                     596018 non-null  int64  
 7   Scr_bytes_ratio                   596018 non-null  float64
 8   Avg_ideal_time                    596018 non-null  float64
 9   byte_rate                         596018 non-null  float64
 10  Service_ssh                       596018 non-null  int64  
 11  is_with_payload                   596018 non-null  b

In [17]:
# Combine selected features from RFE and MRMR
RF_MRMR_data=pd.concat([selected_features_rfe,mrmr_data],axis=1)


In [18]:
RF_MRMR_data = RF_MRMR_data.loc[:,~RF_MRMR_data.columns.duplicated()]

In [19]:
RF_MRMR_data .info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 596018 entries, 0 to 596021
Data columns (total 57 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   Timestamp                         596018 non-null  float64
 1   Scr_port                          596018 non-null  int64  
 2   Des_port                          596018 non-null  int64  
 3   Duration                          596018 non-null  float64
 4   Scr_bytes                         596018 non-null  int64  
 5   Des_bytes                         596018 non-null  int64  
 6   Conn_state                        596018 non-null  int64  
 7   is_syn_only                       596018 non-null  bool   
 8   is_with_payload                   596018 non-null  bool   
 9   FIN or RST                        596018 non-null  bool   
 10  Scr_pkts                          596018 non-null  int64  
 11  Scr_ip_bytes                      596018 non-null  i

In [20]:
col=RF_MRMR_data.columns

In [21]:
print(col)

Index(['Timestamp', 'Scr_port', 'Des_port', 'Duration', 'Scr_bytes',
       'Des_bytes', 'Conn_state', 'is_syn_only', 'is_with_payload',
       'FIN or RST', 'Scr_pkts', 'Scr_ip_bytes', 'Des_pkts', 'Des_ip_bytes',
       'anomaly_alert', 'total_bytes', 'total_packet', 'paket_rate',
       'byte_rate', 'Scr_packts_ratio', 'Des_pkts_ratio', 'Scr_bytes_ratio',
       'Des_bytes_ratio', 'Avg_user_time', 'Avg_nice_time', 'Avg_system_time',
       'Avg_ideal_time', 'Avg_num_cswch/s', 'std_num_cswch/s',
       'OSSEC_alert_level', 'Login_attempt', 'Process_activity',
       'read_write_physical.process', 'Protocol_encoded', 'Service_coap',
       'Service_dns', 'Service_modbus', 'Service_mqtt', 'Service_other',
       'Service_echo', 'missed_bytes', 'File_activity', 'Avg_kbmemused',
       'is_privileged', 'Service_ssh', 'Service_imap', 'Std_ideal_time',
       'Service_mysql', 'Service_simple_service_discovery', 'Service_http',
       'Std_wtps', 'Service_websocket', 'is_SYN_with_RST', 'Avg_

CLASS BALANCING

In [24]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Assuming 'X' is your feature matrix and 'y' is your target variable
X_normalized = StandardScaler().fit_transform(X)

# Split the dataset into train and test sets with an 80:20 ratio
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)

ROS

In [25]:
from imblearn.over_sampling import RandomOverSampler

# Apply Random Oversampling to the training set
ros = RandomOverSampler(sampling_strategy='auto', random_state=42)
X_resampled, y_resampled = ros.fit_resample(X, y)

In [26]:
y_resampled.value_counts()

1    336599
0    336599
Name: class3, dtype: int64

BRFC

In [85]:
from imblearn.ensemble import BalancedRandomForestClassifier

# Create a BalancedRandomForestClassifier
brfc = BalancedRandomForestClassifier(n_estimators=100, random_state=42)

# Fit the classifier on the original imbalanced data
brfc.fit(X_train, y_train)

# Make predictions on the test data
y_pred = brfc.predict(X_test)

c:\Users\Varshini S\AppData\Local\Programs\Python\Python39\lib\site-packages\imblearn\ensemble\_forest.py:576: FutureWarning: The default of `sampling_strategy` will change from `'auto'` to `'all'` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `'all'` to silence this warning and adopt the future behaviour.
  warn(
c:\Users\Varshini S\AppData\Local\Programs\Python\Python39\lib\site-packages\imblearn\ensemble\_forest.py:588: FutureWarning: The default of `replacement` will change from `False` to `True` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `True` to silence this warning and adopt the future behaviour.
  warn(
c:\Users\Varshini S\AppData\Local\Programs\Python\Python39\lib\site-packages\imblearn\ensemble\_forest.py:600: FutureWarning: The default of `bootstrap` will change from `True` to `False` in version 0.13. This change will follow the implementation proposed in the origina

BASE MODEL

In [100]:
from sklearn.ensemble import RandomForestClassifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

In [101]:
y_pred=rf_classifier.predict(X_test)

In [102]:
#Evaluate
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, cohen_kappa_score
# Calculate Precision
precision = precision_score(y_test, y_pred, average='weighted')
# Calculate Recall
recall = recall_score(y_test, y_pred, average='weighted')
# Calculate F1 Score
f1 = f1_score(y_test, y_pred, average='weighted')

# Calculate Accuracy
accuracy = accuracy_score(y_test, y_pred)
# Calculate Cohen's Kappa Score
kappa = cohen_kappa_score(y_test, y_pred)
# Print the evaluation metrics
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Accuracy:", accuracy)
print("Cohen's Kappa Score:", kappa)


Precision: 0.9993208713592003
Recall: 0.9993204926009195
F1 Score: 0.9993204495131762
Accuracy: 0.9993204926009195
Cohen's Kappa Score: 0.9986174928763282


BASE+ BRFC

In [86]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, cohen_kappa_score

# Calculate Precision
precision = precision_score(y_test, y_pred, average='weighted')

# Calculate Recall
recall = recall_score(y_test, y_pred, average='weighted')

# Calculate F1 Score
f1 = f1_score(y_test, y_pred, average='weighted')

# Calculate Accuracy
accuracy = accuracy_score(y_test, y_pred)

# Calculate Cohen's Kappa Score
kappa = cohen_kappa_score(y_test, y_pred)

# Print the evaluation metrics
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Accuracy:", accuracy)
print("Cohen's Kappa Score:", kappa)

Precision: 0.9993961132217558
Recall: 0.9993959934230395
F1 Score: 0.9993959706122779
Accuracy: 0.9993959934230395
Cohen's Kappa Score: 0.9987711674315647


In [87]:
X_train_resampled, X_test_resampled, y_train_resampled, y_test_resampled = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

BASE + ROS

In [88]:
from sklearn.ensemble import RandomForestClassifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train_resampled, y_train_resampled)

RandomForestClassifier(random_state=42)

In [89]:
y_pred_resampled=rf_classifier.predict(X_test_resampled)

In [90]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, cohen_kappa_score
# Calculate Precision
precision = precision_score(y_test_resampled, y_pred_resampled, average='weighted')

# Calculate Recall
recall = recall_score(y_test_resampled, y_pred_resampled, average='weighted')

# Calculate F1 Score
f1 = f1_score(y_test_resampled, y_pred_resampled, average='weighted')

# Calculate Accuracy
accuracy = accuracy_score(y_test_resampled, y_pred_resampled)

# Calculate Cohen's Kappa Score
kappa = cohen_kappa_score(y_test_resampled, y_pred_resampled)

# Print the evaluation metrics
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Accuracy:", accuracy)
print("Cohen's Kappa Score:", kappa)

Precision: 0.9995173979348749
Recall: 0.9995172311348782
F1 Score: 0.9995172306706112
Accuracy: 0.9995172311348782
Cohen's Kappa Score: 0.9990344517083385


RF_MRMR

In [91]:
X_train_rfmrmr, X_test_rfmrmr, y_train_rfmrmr, y_test_rfmrmr = train_test_split(RF_MRMR_data, y, test_size=0.2, random_state=42)

In [92]:
rf_classifier.fit(X_train_rfmrmr, y_train_rfmrmr)

RandomForestClassifier(random_state=42)

In [93]:
y_pred_rfmrmr=rf_classifier.predict(X_test_rfmrmr)

In [94]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, cohen_kappa_score

# Calculate Precision
precision = precision_score(y_test_rfmrmr, y_pred_rfmrmr, average='weighted')

# Calculate Recall
recall = recall_score(y_test_rfmrmr, y_pred_rfmrmr, average='weighted')

# Calculate F1 Score
f1 = f1_score(y_test_rfmrmr, y_pred_rfmrmr, average='weighted')

# Calculate Accuracy
accuracy = accuracy_score(y_test_rfmrmr, y_pred_rfmrmr)

# Calculate Cohen's Kappa Score
kappa = cohen_kappa_score(y_test_rfmrmr, y_pred_rfmrmr)

# Print the evaluation metrics
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Accuracy:", accuracy)
print("Cohen's Kappa Score:", kappa)

Precision: 0.999572270089527
Recall: 0.9995721620079863
F1 Score: 0.9995721472789827
Accuracy: 0.9995721620079863
Cohen's Kappa Score: 0.999129582718914


RF_MRMR+ROS

In [27]:
X_resampled_rfmrmr, y_resampled_rfmrmr = ros.fit_resample(RF_MRMR_data, y)

In [28]:
X_train_rfmrmr_resampled, X_test_rfmrmr_resampled, y_train_rfmrmr_resampled, y_test_rfmrmr_resampled = train_test_split(X_resampled_rfmrmr, y_resampled_rfmrmr, test_size=0.2, random_state=42)

In [30]:
from sklearn.ensemble import RandomForestClassifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train_rfmrmr_resampled, y_train_rfmrmr_resampled)

RandomForestClassifier(random_state=42)

In [31]:
y_pred_rfmrmr_resampled=rf_classifier.predict(X_test_rfmrmr_resampled)

In [32]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, cohen_kappa_score

# Calculate Precision
precision = precision_score(y_test_rfmrmr_resampled, y_pred_rfmrmr_resampled, average='weighted')

# Calculate Recall
recall = recall_score(y_test_rfmrmr_resampled, y_pred_rfmrmr_resampled, average='weighted')

# Calculate F1 Score
f1 = f1_score(y_test_rfmrmr_resampled, y_pred_rfmrmr_resampled, average='weighted')

# Calculate Accuracy
accuracy = accuracy_score(y_test_rfmrmr_resampled, y_pred_rfmrmr_resampled)

# Calculate Cohen's Kappa Score
kappa = cohen_kappa_score(y_test_rfmrmr_resampled, y_pred_rfmrmr_resampled)

# Print the evaluation metrics
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Accuracy:", accuracy)
print("Cohen's Kappa Score:", kappa)

Precision: 0.9995395986121426
Recall: 0.9995395127748069
F1 Score: 0.9995395124646949
Accuracy: 0.9995395127748069
Cohen's Kappa Score: 0.9990790159316739


RF_MRMR + BRFC

In [103]:
X_train_rfmrmr, X_test_rfmrmr, y_train_rfmrmr, y_test_rfmrmr = train_test_split(RF_MRMR_data, y, test_size=0.2, random_state=42)

# Fit the classifier on the original imbalanced data
brfc.fit(X_train_rfmrmr,y_train_rfmrmr)


c:\Users\Varshini S\AppData\Local\Programs\Python\Python39\lib\site-packages\imblearn\ensemble\_forest.py:576: FutureWarning: The default of `sampling_strategy` will change from `'auto'` to `'all'` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `'all'` to silence this warning and adopt the future behaviour.
  warn(
c:\Users\Varshini S\AppData\Local\Programs\Python\Python39\lib\site-packages\imblearn\ensemble\_forest.py:588: FutureWarning: The default of `replacement` will change from `False` to `True` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `True` to silence this warning and adopt the future behaviour.
  warn(
c:\Users\Varshini S\AppData\Local\Programs\Python\Python39\lib\site-packages\imblearn\ensemble\_forest.py:600: FutureWarning: The default of `bootstrap` will change from `True` to `False` in version 0.13. This change will follow the implementation proposed in the origina

BalancedRandomForestClassifier(random_state=42)

In [104]:
y_pred_rfmrmr_brfc= brfc.predict(X_test_rfmrmr)

In [105]:
# Evaluate
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, cohen_kappa_score

# Calculate Precision
precision = precision_score(y_test_rfmrmr, y_pred_rfmrmr_brfc, average='weighted')

# Calculate Recall
recall = recall_score(y_test_rfmrmr, y_pred_rfmrmr_brfc, average='weighted')

# Calculate F1 Score
f1 = f1_score(y_test_rfmrmr, y_pred_rfmrmr_brfc, average='weighted')

# Calculate Accuracy
accuracy = accuracy_score(y_test_rfmrmr, y_pred_rfmrmr_brfc)

# Calculate Cohen's Kappa Score
kappa = cohen_kappa_score(y_test_rfmrmr,y_pred_rfmrmr_brfc)

# Print the evaluation metrics
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Accuracy:", accuracy)
print("Cohen's Kappa Score:", kappa)


Precision: 0.9995302867305433
Recall: 0.9995302171068084
F1 Score: 0.9995302035468596
Accuracy: 0.9995302171068084
Cohen's Kappa Score: 0.9990442582839941


In [33]:
from joblib import dump
dump(rf_classifier,"2nd_Model.joblib")

['2nd_Model.joblib']